# Scripting to Cloud Based AI via API

## Anthropic Test Case

First we need to install our python application extension

We will discuss each one as we go through.

We will also be using a sample pipline with the following steps:


1.   Pull data from existing system -> SF Chat Transcripts
2.   Format Data and upload to Google Colab
3. Upload the excel file into a Panda Data Scheme using Python
4. Scrub data through several passes to remove uneeded html characters and PII
5. Use the data as a prompt when interacting with AI (Summarize and coach the convo)
6. Analyze the AI Output JSON and format it into the PD
7. Export the PD as a single file

This shows what can be done as a POC to help define and design a fully automated AI pipline where these steps and triggers happen automatically



In [ ]:
!pip install anthropic
!pip install pandas
!pip install google-colab

Next we can start scripting to against the response engine

In [ ]:
import anthropic
client = anthropic.Anthropic(
    api_key="APIKEY"
)
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
if message.content:
      print("\n" + "=" * 50)
      print("Claude's Response:")
      print("=" * 50)
      print(message.content[0].text)
      print("=" * 50 + "\n")
else:
      print("No response generated.")

Changing the prompt and parent prompt as an example - note the token cost on Anthropic

In [ ]:
import anthropic
client = anthropic.Anthropic(
    api_key="APIKEY"
)
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a helpful assistant",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are some of the tenants to good project management?"
                }
            ]
        }
    ]
)
if message.content:
      print("\n" + "=" * 50)
      print("Claude's Response:")
      print("=" * 50)
      print(message.content[0].text)
      print("=" * 50 + "\n")
else:
      print("No response generated.")

## Scripting Bulk Requests

Now that we have successfully created a connection to Claude - we can script a bulk request by uploading a sample data set, teaching  the AI how to access the data for each prompt, and then looping through the data set.

### Step 1

In this example lets walk through uploading our exel file of data. If needed we can walk through how to export this data from Salesforce or other tools (for Salesforce I like to use workbench) but for now we will assume we already have data to use.

We are going to use whats called a panda data structure, these are popular dimmensial type arrays that are frequently used in python for large data manipulation and data science. You can read more about them [here](https://pandas.pydata.org/docs/index.html) (or ask AI).


In [ ]:
# Import necessary libraries
import pandas as pd  # pandas is used for data manipulation and analysis
from google.colab import files  # This module helps with file uploads in Google Colab
import io  # io provides Python's main facilities for dealing with various types of I/O

# Define a function to upload and load an Excel file
def upload_and_load_excel():
    try:
        # Prompt the user to upload a file
        print("Please upload your Excel file.")

        # Use Google Colab's file upload feature
        # This will open a file picker for the user to select a file
        uploaded = files.upload()

        # Check if a file was actually uploaded
        if not uploaded:
            print("No file was uploaded.")
            return None  # Return None if no file was uploaded

        # Get the name of the uploaded file
        # 'next(iter(uploaded))' gets the first (and only) key from the uploaded dictionary
        file_name = next(iter(uploaded))
        print(f"File '{file_name}' has been uploaded successfully.")

        # Read the uploaded Excel file into a pandas DataFrame
        # BytesIO creates a file-like object in memory, which pandas can read
        df = pd.read_excel(io.BytesIO(uploaded[file_name]), engine='openpyxl')

        # Check if the DataFrame has at least two columns
        # df.shape returns a tuple with (number of rows, number of columns)
        if df.shape[1] < 2:
            # Raise an error if there aren't enough columns
            raise ValueError("The Excel file must contain at least two columns.")

        # Print information about the loaded data
        print(f"Successfully loaded data with {df.shape[0]} rows and {df.shape[1]} columns.")

        # Return the DataFrame for further use
        return df

    except Exception as e:
        # If any error occurs during the process, print the error message
        print(f"An error occurred: {e}")
        return None  # Return None to indicate that the operation failed

# Call the function to upload and load the Excel file
print("Starting the Excel file upload and load process...")
df = upload_and_load_excel()

# Check if the DataFrame was successfully created
if df is not None:
    # If df is not None, it means the file was successfully loaded
    print("\nFirst few rows of the data:")
    print(df.head())  # Display the first 5 rows of the DataFrame

    print("\nDataFrame info:")
    df.info()  # Print a concise summary of the DataFrame
else:
    # If df is None, it means there was an error in loading the file
    print("Failed to load the data. Please check the error message above and try again.")

# Note: After running this code, 'df' will contain your Excel data as a pandas DataFrame,
# which you can use for further analysis or processing.

We can test that our data is in a Panda DF by printing it (note that code you have run previously in the notebook and variables that are output are available to you as you work)


In [ ]:
print(df)

### Step 2

Now that we have our data in our system memory in the form of a Data Frame, we can begin to manipulate it, since we are using a public AI and there is a per token cost, lets see if we can reduce the number of tokens. Also since this is company information and I am not in a closed environment I need to ensure I have removed all PII.

Lets import some public libraries that clean out CC, email and other payment information. These libraries are publicaly available for exercises like this. You can read more about them here. [Sanityze](https://pypi.org/project/sanitize/#description)

In [ ]:
!pip install bleach
!pip install xlsxwriter

In [ ]:
import pandas as pd
import bleach
import re

def clean_text(text):
    if pd.isna(text):
        return text

    # Convert to string if it's not already
    text = str(text)

    # Use bleach to clean HTML tags
    cleaned = bleach.clean(text, tags=[], strip=True)

    # Remove extra whitespace and newlines
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Remove any remaining HTML entities
    cleaned = bleach.clean(cleaned, strip=True)

    return cleaned

def clean_dataframe(df, columns=None):
    # If no columns specified, clean all object (string) columns
    if columns is None:
        columns = df.select_dtypes(include=['object']).columns

    # Apply the clean_text function to specified columns
    for col in columns:
        df[col] = df[col].apply(clean_text)

    return df

# Load your DataFrame (assuming you've already done this)
# df = pd.read_excel('your_file.xlsx')

# Clean the DataFrame
bleacheddf = clean_dataframe(df)

# If you want to clean only specific columns:
# newdf = clean_dataframe(df, columns=['column1', 'column2'])

print(bleacheddf.head())

At any point we can convert our Panda data frame back to an excel or csv, in this case I am going to export the bleachedDF so we can compare to our original.


In [47]:
import pandas as pd
from io import BytesIO
from google.colab import files

def export_df_to_excel(bleacheddf, filename='dataframe_export.xlsx'):
    """
    Export a pandas DataFrame to an Excel file and download it in Google Colab.

    Parameters:
    bleacheddf (pandas.DataFrame): The DataFrame to export
    filename (str): The name of the file to be downloaded (default is 'dataframe_export.xlsx')
    """
    # Create a BytesIO object
    output = BytesIO()

    # Create a Pandas Excel writer using XlsxWriter as the engine
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        # Write the DataFrame to the Excel file
        bleacheddf.to_excel(writer, sheet_name='Sheet1', index=False)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        # Set the column widths (adjust as needed)
        for i, col in enumerate(bleacheddf.columns):
            column_len = max(bleacheddf[col].astype(str).str.len().max(), len(col) + 2)
            worksheet.set_column(i, i, column_len)

    # Reset the buffer position to the beginning
    output.seek(0)

    # Write the BytesIO content to the file
    with open(filename, 'wb') as f:
        f.write(output.getvalue())

    # Use google.colab.files.download to trigger the file download
    files.download(filename)

# Example usage
# Create a sample DataFrame
data = bleacheddf
bleacheddf = pd.DataFrame(data)

# Export the DataFrame to an Excel file
export_df_to_excel(bleacheddf, 'example_export.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

As you can see we have already made the input much clearer and cleaner and reduced the size of our input text (or character count). Sometimes it can be helpful to include extra data in your prompt but not in this case.


### Step 3

At this point we can remove names for extra security (this would be another step in the pipline or another pass in the pipeline mentioned in the intro)

We need to install another library to remove names (we have only removed HTML)


In [ ]:
!pip install spacy download en_core_web_sm

Now we can scrape our chat for our names. Don't worry the AI can still understand who is who. 😀


In [ ]:
import pandas as pd
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Assuming newdf is your DataFrame and 'column_name' is the name of the column you want to process
# Replace 'column_name' with the actual name of your column
column_name = 'Body'

# Function to anonymize text by replacing person names with [REDACTED]
def anonymize_text(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            text = text.replace(ent.text, "[REDACTED]")
    return text

# Apply the anonymization function to the specified column
bleacheddf[column_name] = bleacheddf[column_name].apply(anonymize_text)

# Print the DataFrame to verify changes
print(bleacheddf)

You can run the export module from above to see the new output - since we are not creating a new DF and just over writting the previous DF we do not need to modify the code. That module can be modified so any DF can be exported


Let's add one more pass to remove unused tokens and articles and line breaks (this makes the prompt cheaper)

In [39]:
import pandas as pd
import re

# Sample DataFrame creation (assuming you already have a DataFrame)
# data = {'ID': [1, 2], 'Transcript': ['<p>Chat transcript 1</p>', '<p>Chat transcript 2</p>']}
# Function to clean and format a single chat transcript
def clean_transcript(Body):
    # Remove HTML tags
    cleaned_transcript = re.sub(r'<[^>]+>', '', Body)

    # Extract and format dialogues
    dialogues = re.findall(r'\(.*?\)\s(.*?):\s(.*?)\s*(?:<br>|\Z)', cleaned_transcript, re.DOTALL)

    formatted_transcript = []
    for agent, text in dialogues:
        formatted_transcript.append(f"{agent}: {text}")

    return "\n".join(formatted_transcript)

# Applying the function to the DataFrame
def process_transcripts(bleacheddf, Body):
    bleacheddf[Body] = bleacheddf[Body].apply(clean_transcript)
    return bleacheddf

# Assuming your DataFrame is named 'df' and the column with transcripts is named 'Transcript'
bleacheddf = process_transcripts(bleacheddf, 'Body')
print(bleacheddf)


                                                Body                SFID
0  Chat Origin: United States Agent [REDACTED]( 7...  5707S0000012U7jQAE
1  [REDACTED]: Denmark Agent [REDACTED]( 28s ) [R...  5707S00000124QQQAY
2  [REDACTED]: France Agent [REDACTED] D( 19m 37s...  5707S0000012ES7QAM
3  [REDACTED]: United States Agent [REDACTED]( 31...  5707S0000012AKOQA2
4  [REDACTED]: United States Agent Mj B( 5m 54s )...  5707S0000012F12QAE
5  [REDACTED]: United States Agent [REDACTED]( 48...  5707S0000012fh9QAA
6  Chat Origin: United States Agent [REDACTED]( 1...  5707S0000012eZYQAY
7  [REDACTED]: United States Agent [REDACTED]( 2m...  5707S00000129qnQAA
8  [REDACTED]: United States Agent [REDACTED]( 22...  5707S0000012CctQAE


### Step 4

At this point we are safe to process through and loop our data as the prompt

Let's make it easy and set our system prompt first.


In [43]:
# System prompt
SYSTEM_PROMPT = """You are an expert at understanding conversations between travel agents and customers.
When given a transcript of a conversation, you will respond with a JSON structure using the following format:
{
  "Summary": <string>,
  "Actions": <array of strings>,
  "Intents": <array of strings>,
  "Sentiment": <string>,
  "Arranger": <string>,
  "FCR": <string>,
  "Reason": <array of strings>,
  "Coaching": <string>
}
The fields can be described as follows:
Summary: a concise summary of the conversation

Actions: concise multi word phrases joined by a - character, like "search-flight", "lookup-itinerary", "book-hotel", "cancel-flight". these describe actions that the agent took while in the conversation

Intents: concise multi word phrases joined by a - character, like "quote-flight", "cancel-flight", "change-hotel", "request-invoice". these describe the intent of the customer answering the question of why did they reach out and contact the travel agent

Sentiment: this describes the attitude of the customer by the end of the conversation. possible values include "positive" for when they are happy or satisfied, "negative" for when they are unhappy or unsatisfied and finally "neutral" if there is uncertainty on how they are feeling.

Arranger: this describes in a single word "arranger" or "self" if the person contacting is requesting assistance on behalf of themselves or someone else. If the customer needs help for anyone but themselves it should be labeled as arranger

FCR: First chat resolution, in a single word (yes/no) please identify if the caller's issue was resolved on the call or if they will require additional support to resolve their issue

Reason: Pick applicable items from the following list, pick only items from this list but pick all applicable items from this list. [Book Flight, Book Hotel, Book Car Rental, Change/Cancel Existing Reservation, Request Travel Policy Information, Expense Reimbursement Question, Invoice or Receipt Request]

Coaching: This is the single best coaching you can recommend to the agent based on the conversation. Must be a single sentence.
"""

In [45]:
import pandas as pd
import anthropic
import json

client = anthropic.Anthropic(
    api_key="ApiKey"
)

# System prompt remains the same

def analyze_conversation(transcript):
    try:
        response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
            system=SYSTEM_PROMPT,
            messages=[
                {
                    "role": "user",
                    "content": transcript
                }
            ]
        )

        # Parse the JSON response
        json_response = json.loads(response.content[0].text)
        return json_response
    except json.JSONDecodeError:
        print(f"Error decoding JSON from response: {response.content[0].text}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Assuming you have a DataFrame named 'df'
# Loop through each row in the DataFrame
for index, row in bleacheddf.iterrows():
    transcript = row['Body']  # Assuming the column is named 'Body'

    analysis = analyze_conversation(transcript)

    if analysis:
        # Update the DataFrame with the analysis results
        bleacheddf.at[index, 'Summary'] = analysis.get('Summary', '')
        bleacheddf.at[index, 'Actions'] = ', '.join(analysis.get('Actions', []))
        bleacheddf.at[index, 'Intents'] = ', '.join(analysis.get('Intents', []))
        bleacheddf.at[index, 'Sentiment'] = analysis.get('Sentiment', '')
        bleacheddf.at[index, 'Arranger'] = analysis.get('Arranger', '')
        bleacheddf.at[index, 'FCR'] = analysis.get('FCR', '')
        bleacheddf.at[index, 'Reason'] = ', '.join(analysis.get('Reason', []))
        bleacheddf.at[index, 'Coaching'] = analysis.get('Coaching', '')
    else:
        print(f"Failed to analyze row {index}")

    # Optional: Print progress
    if index % 10 == 0:
        print(f"Processed {index} rows")

# Optionally, you can save the updated DataFrame
# df.to_csv('updated_travel_data.csv', index=False)

print("Processing complete!")

Processed 0 rows
Processing complete!


Now we can print our data to verify before export

In [46]:
print(bleacheddf)

                                                Body                SFID  \
0  Chat Origin: United States Agent [REDACTED]( 7...  5707S0000012U7jQAE   
1  [REDACTED]: Denmark Agent [REDACTED]( 28s ) [R...  5707S00000124QQQAY   
2  [REDACTED]: France Agent [REDACTED] D( 19m 37s...  5707S0000012ES7QAM   
3  [REDACTED]: United States Agent [REDACTED]( 31...  5707S0000012AKOQA2   
4  [REDACTED]: United States Agent Mj B( 5m 54s )...  5707S0000012F12QAE   
5  [REDACTED]: United States Agent [REDACTED]( 48...  5707S0000012fh9QAA   
6  Chat Origin: United States Agent [REDACTED]( 1...  5707S0000012eZYQAY   
7  [REDACTED]: United States Agent [REDACTED]( 2m...  5707S00000129qnQAA   
8  [REDACTED]: United States Agent [REDACTED]( 22...  5707S0000012CctQAE   

                                             Summary  \
0  A customer arrived in San Francisco and discov...   
1  A customer contacted the travel agent seeking ...   
2  The customer contacted the agent regarding a c...   
3  The customer

Obviously this is a bit hard to read so lets convert this data into an excel friendly version

In [ ]:
import pandas as pd
from io import BytesIO
from google.colab import files

def export_df_to_excel(bleacheddf, filename='dataframe_export.xlsx'):
    """
    Export a pandas DataFrame to an Excel file and download it in Google Colab.

    Parameters:
    bleacheddf (pandas.DataFrame): The DataFrame to export
    filename (str): The name of the file to be downloaded (default is 'dataframe_export.xlsx')
    """
    # Create a BytesIO object
    output = BytesIO()

    # Create a Pandas Excel writer using XlsxWriter as the engine
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        # Write the DataFrame to the Excel file
        bleacheddf.to_excel(writer, sheet_name='Sheet1', index=False)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        # Set the column widths (adjust as needed)
        for i, col in enumerate(bleacheddf.columns):
            column_len = max(bleacheddf[col].astype(str).str.len().max(), len(col) + 2)
            worksheet.set_column(i, i, column_len)

    # Reset the buffer position to the beginning
    output.seek(0)

    # Write the BytesIO content to the file
    with open(filename, 'wb') as f:
        f.write(output.getvalue())

    # Use google.colab.files.download to trigger the file download
    files.download(filename)

# Example usage
# Create a sample DataFrame
data = bleacheddf
bleacheddf = pd.DataFrame(data)

# Export the DataFrame to an Excel file
export_df_to_excel(bleacheddf, 'final.xlsx')

And there we go! We have a fully automated pipline - if all tested well, you can go download a new set of chats, make sure there are only two columns (or three can't remember haha) and that the transcript column is named Body.

Note: all these paramaters can be changed in the code, thats just how the code is functioning now

This is the EXACT workbench query you can use to pull your data - you can update the limits, add filters and change dates etc

```
SELECT Body,Id FROM LiveChatTranscript WHERE Language__c = 'English' AND OperatorMessageCount > 1 AND CreatedDate > 2024-01-08T00:00:00Z LIMIT 5
```

